In [43]:
import pandas as pd
import json
import dask as dd
import numpy as np
import re

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

with open(r'C:\Users\leoso\Projects\books_dict.txt', 'r') as file:
    raw_data = json.load(file)

raw_data = pd.DataFrame(raw_data).transpose()

drop_list = [
    'Idioma',
    'Tamanho do arquivo',
    'Leitura de texto',
    'Leitor de tela',
    'Configuração de fonte',
    'Dicas de vocabulário',
    'Dimensões',
    'Idade de leitura',
    'ISBN da fonte dos números de páginas',
    'Livro de bolso',
    'Encadernação com grampos',
    'Quantidade de dispositivos em que é possível ler este eBook ao mesmo tempo',
    'Acessório',
    'X-Ray',
    'Encadernação de livro didático',
    'Revista',
    'Edição econômica',
    'Capa flexível',
    'Acabamento especial',
    'Impressão sob demanda (capa comum)',
    'Formato',
    'Número de páginas',
    'Page Ranking',
    'Page',
    'Capa comum',
    'Capa dura',
    'Cat1'
]

rename_dict = {
    'Overall Ranking': 'Ranking Feminismo',
    'Avaliação_média': 'Avaliação média',
    'Rank_Cat1': 'Ranking Livros',
    'Cat2': 'Categoria 1',
    'Rank_Cat2': 'Ranking Categoria 1',
    'Cat3': 'Categoria 2',
    'Rank_Cat3': 'Ranking Categoria 2',
    'Cat4': 'Categoria 3',
    'Rank_Cat4': 'Ranking Categoria 3',
    'Tipo de livro 1': 'Tipo Livro 1',
    'Tipo de livro 2': 'Tipo Livro 2',
    'Valor 1': 'Preço 1',
    'Valor 2': 'Preço 2'
}

raw_data.drop(drop_list, axis = 1, inplace=True)
raw_data.rename(rename_dict, axis = 1, inplace = True)

pattern = re.compile(
    r'^(?P<editora>[^;(]+?)?'               # 1) Publisher name (anything up to a semicolon or parenthesis)
    r'\s*'                                  #    possible spaces
    r'(?:;\s*(?P<edicao>[^()]+))?'          # 2) Optional semicolon + edition text (capture in group 'edicao')
    r'\s*'                                  #    possible spaces
    r'(?:\(\s*(?P<data>\d{1,2}\s*'          # 3) Optional parentheses containing date
    r'(?:janeiro|fevereiro|março|abril|maio|junho|'
    r'julho|agosto|setembro|outubro|novembro|dezembro)\s*\d{4})\s*\))?'
    r'\s*$'                                 #    end of string (allowing trailing spaces)
)

def parse_editora(value):
    # Handle NaN or empty strings
    if pd.isna(value) or not isinstance(value, str) or not value.strip():
        return pd.Series({'Editora': '', 'Edição': '', 'Data de Publicação': ''})
    
    match = pattern.match(value.strip())
    if match:
        return pd.Series({
            'Editora': match.group('editora') or '',
            'Edição': match.group('edicao') or '',
            'Data de Publicação': match.group('data') or ''
        })
    else:
        # If it doesn't match any pattern, treat as blank or NaN as needed
        return pd.Series({'Editora': '', 'Edição': '', 'Data de Publicação': ''})

raw_data[['Editora','Edição','Data de Publicação']] = raw_data['Editora'].apply(parse_editora)

month_map = {
    'janeiro': '01',
    'fevereiro': '02',
    'março': '03',
    'abril': '04',
    'maio': '05',
    'junho': '06',
    'julho': '07',
    'agosto': '08',
    'setembro': '09',
    'outubro': '10',
    'novembro': '11',
    'dezembro': '12'
}

def to_ddmmyyyy(date_str):
    """
    Convert a date string in the format 'd mmmm yyyy' (Portuguese month names)
    to 'dd/mm/yyyy'. If the input is blank or NaN, return ''.
    """
    # Check for NaN, None, or empty
    if not isinstance(date_str, str) or not date_str.strip():
        return ''
    
    # Safely split into day, month name, year
    parts = date_str.strip().split()
    if len(parts) != 3:
        return ''  # or handle errors differently
    
    day_str, month_name, year_str = parts
    
    # Normalize the month name to lowercase, map to month number
    month_name = month_name.lower()
    if month_name not in month_map:
        return ''  # unknown month name
    
    # Zero-pad day if needed
    day_str = day_str.zfill(2)
    
    return f"{day_str}/{month_map[month_name]}/{year_str}"

raw_data['Data'] = raw_data['Data de Publicação'].apply(to_ddmmyyyy)

raw_data = raw_data[[
    'Nome',
    'Link',
    'Editora',
    'Data',
    'Edição',
    'Autor 1',
    'Autor 2',
    'Ranking Feminismo',
    'Ranking Livros',
    'Categoria 1',
    'Ranking Categoria 1',
    'Categoria 2',
    'Ranking Categoria 2',
    'Categoria 3',
    'Ranking Categoria 3',
    'Tipo Livro 1',
    'Preço 1',
    'Tipo Livro 2',
    'Preço 2',
    'ASIN',
    'ISBN-10',
    'ISBN-13',
]]

In [55]:
raw_data.to_excel(r'C:\Users\leoso\Projects\Livros Amazon.xlsx')